# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. These are combined pluvial and fluvial (undefended) flood layers whereby whichever flood level was higher was retained.
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box, Polygon
from shapely.ops import unary_union, linemerge, transform
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
from GOSTnets import load_osm as losm
import importlib

In [7]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [8]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [9]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Prepare OSM driving network.
Travel measured in length (meters).

#### Ensure all targeted roads are changed to tertiary. 

In [10]:
flood50 = gpd.read_file("C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/scratch.gdb", layer="PFU_1in50")
flood50.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2459356 entries, 0 to 2459355
Data columns (total 4 columns):
 #   Column        Dtype   
---  ------        -----   
 0   PFU_1in50     float64 
 1   Shape_Length  float64 
 2   Shape_Area    float64 
 3   geometry      geometry
dtypes: float64(3), geometry(1)
memory usage: 75.1 MB


In [11]:
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime_pre-project.pickle")

In [12]:
edges = gn.edge_gdf_from_graph(gTime)

In [13]:
nodes = os.path.join(out_pth, "gTime_node_pre-project.csv")
nodes = pd.read_csv(nodes)
print(nodes.info())
print(edges.info())

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871182 entries, 0 to 2871181
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   node_ID     int64  
 2   highway     object 
 3   ref         object 
 4   y           float64
 5   x           float64
 6   geometry    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 153.3+ MB
None
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6203383 entries, 0 to 6203382
Data columns (total 22 columns):
 #   Column      Dtype   
---  ------      -----   
 0   stnode      int64   
 1   endnode     int64   
 2   name        object  
 3   access      object  
 4   maxspeed    object  
 5   length      float64 
 6   bridge      object  
 7   time        float64 
 8   oneway      bool    
 9   lanes       float64 
 10  tunnel      object  
 11  area        object  
 12  landuse     object  
 13  service     object  
 14  ref         object  
 15  mode        object  
 16  

In [14]:
# Spatial join should be on projected GDFs.
edges = edges.to_crs("EPSG:31028")
flood50 = flood50.to_crs("EPSG:31028")
edges.crs == flood50.crs

True

In [15]:
floodjoin = gpd.sjoin_nearest(edges, flood50, how="left", max_distance=3) 
print(floodjoin.info())
print(floodjoin)

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6508966 entries, 0 to 6203382
Data columns (total 26 columns):
 #   Column        Dtype   
---  ------        -----   
 0   stnode        int64   
 1   endnode       int64   
 2   name          object  
 3   access        object  
 4   maxspeed      object  
 5   length        float64 
 6   bridge        object  
 7   time          float64 
 8   oneway        bool    
 9   lanes         float64 
 10  tunnel        object  
 11  area          object  
 12  landuse       object  
 13  service       object  
 14  ref           object  
 15  mode          object  
 16  Unnamed: 0    int64   
 17  junction      object  
 18  highway       object  
 19  width         object  
 20  osmid         int64   
 21  geometry      geometry
 22  index_right   float64 
 23  PFU_1in50     float64 
 24  Shape_Length  float64 
 25  Shape_Area    float64 
dtypes: bool(1), float64(7), geometry(1), int64(4), object(13)
memory usage: 1.3+ GB
None
     

In [16]:
# How many nodes experienced flooding?
pc_flooded = floodjoin["PFU_1in50"].count() / len(floodjoin) * 100

print("No flood crossing at node:", floodjoin["PFU_1in50"].isnull().sum(), "locations", end="\n\n")
print("Flood crossing at node:", floodjoin["PFU_1in50"].count(), "locations", end="\n\n")
print("\nPercent flooded:", pc_flooded, "percent", "out of", len(floodjoin), "possible locations")

No flood crossing at node: 5745481 locations

Flood crossing at node: 763485 locations


Percent flooded: 11.729743249542247 percent out of 6508966 possible locations


In [17]:
floodjoin = floodjoin[['stnode', 'endnode', 'time', 'length', 'highway', 'osmid', 'geometry', 'PFU_1in50']]
floodjoin = floodjoin.to_crs("EPSG:4326")
floodjoin.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6508966 entries, 0 to 6203382
Data columns (total 8 columns):
 #   Column     Dtype   
---  ------     -----   
 0   stnode     int64   
 1   endnode    int64   
 2   time       float64 
 3   length     float64 
 4   highway    object  
 5   osmid      int64   
 6   geometry   geometry
 7   PFU_1in50  float64 
dtypes: float64(3), geometry(1), int64(3), object(1)
memory usage: 446.9+ MB


In [18]:
# Fewer errors farther down when using dataframe instead of gdf
floodjoin = pd.DataFrame(floodjoin)

In [19]:
# Save progress.
floodjoin.to_csv(os.path.join(out_pth, 'gTime_edge_pre-project_flood50.csv'))

### Create speed penalties.
Note: Flood depths are in centimeters. FATHOM uses meters, but conversion process to vector required some finessing. 

In [20]:
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'edges', 'exit', 'fiona', 'flood50', 'floodjoin', 'gTime', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'nodes', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']


In [21]:
del flood50, gTime, edges, floodjoin
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'exit', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'nodes', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']


In [22]:
# Alternatively, restart the kernel and reload.
floodjoin = os.path.join(out_pth, "gTime_edge_pre-project_flood50.csv")
floodjoin = pd.read_csv(floodjoin)

In [23]:
# Give a depth to the nodes that don't cross a flood point. 
floodjoin.loc[floodjoin['PFU_1in50'].isnull(), 'PFU_1in50'] = -1
floodjoin["t50"] = 1 # This is the penalty column.
floodjoin.loc[floodjoin['PFU_1in50'] <= 5, 't50'] = 1 # Where no flood crosses, keep the default value (no penalty).
floodjoin.loc[(floodjoin['PFU_1in50'] > 5) & (floodjoin['PFU_1in50'] <= 10), 't50'] = 1.25 # Between 10-30cm flooding, increase travel time by 1.25.
floodjoin.loc[(floodjoin['PFU_1in50'] > 10) & (floodjoin['PFU_1in50'] <= 15), 't50'] = 2
floodjoin.loc[(floodjoin['PFU_1in50'] > 15) & (floodjoin['PFU_1in50'] <= 40), 't50'] = 5
floodjoin.loc[(floodjoin['PFU_1in50'] > 40), 't50'] = 9999
floodjoin

,Unnamed: 0,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50,t50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
3,3,5217543379,358284990,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.0
4,4,5217543379,5329928981,1.232928,17.124,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
6508961,6203378,9165665162,9165665161,4.380800,5.476,path,992024621,LINESTRING (-17.484701707370633 14.74572550266...,-1.0,1.0
6508962,6203379,9201905615,9201918817,0.307620,3.418,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,1.0
6508963,6203380,9201905615,9201918818,0.634050,7.045,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,1.0
6508964,6203381,9201918818,9201905615,0.634050,7.045,residential,996565464,LINESTRING (-17.492688607366233 14.73786330266...,-1.0,1.0


In [24]:
# Turn the penalty column into a flood-affected time column.
floodjoin['t50'] = floodjoin['t50'] * floodjoin['time']
floodjoin

,Unnamed: 0,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50,t50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,2.385144
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,0.769920
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,2.926860
3,3,5217543379,358284990,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,2.385144
4,4,5217543379,5329928981,1.232928,17.124,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.232928
...,...,...,...,...,...,...,...,...,...,...
6508961,6203378,9165665162,9165665161,4.380800,5.476,path,992024621,LINESTRING (-17.484701707370633 14.74572550266...,-1.0,4.380800
6508962,6203379,9201905615,9201918817,0.307620,3.418,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,0.307620
6508963,6203380,9201905615,9201918818,0.634050,7.045,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,0.634050
6508964,6203381,9201918818,9201905615,0.634050,7.045,residential,996565464,LINESTRING (-17.492688607366233 14.73786330266...,-1.0,0.634050


In [25]:
# Save progress.
floodjoin.to_csv(os.path.join(out_pth, 'gTime_edge_pre-project_flood50.csv'))

In [26]:
print("Travel time without inclement weather\n")
print("Mean:", floodjoin['time'].mean(), end="\n")
print("Median:", floodjoin['time'].median(), end='\n\n')
print("\n\nDuring 1 in 50-year flood conditions\n")
print("Mean:", floodjoin['t50'].mean(), end="\n") 
print("Median:", floodjoin['t50'].median()) 

Travel time without inclement weather

Mean: 7.464243950039652
Median: 3.2853600000000003



During 1 in 50-year flood conditions

Mean: 8461.404240362535
Median: 3.5988685714285715


### Convert back to graph object.

In [27]:
# Converting back to graph can cause memory errors. Suggested to restart the kernel and reload the nodes and revised edges at this point.
del floodjoin, nodes
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_24', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'exit', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']


In [28]:
nodes = os.path.join(out_pth, "gTime_node_pre-project.csv")
nodes = pd.read_csv(nodes)
floodjoin = os.path.join(out_pth, "gTime_edge_pre-project_flood50.csv")
floodjoin = pd.read_csv(floodjoin)
print(nodes.info())
print(floodjoin.info())

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871182 entries, 0 to 2871181
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   node_ID     int64  
 2   highway     object 
 3   ref         object 
 4   y           float64
 5   x           float64
 6   geometry    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 153.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6508966 entries, 0 to 6508965
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   Unnamed: 0.1  int64  
 2   stnode        int64  
 3   endnode       int64  
 4   time          float64
 5   length        float64
 6   highway       object 
 7   osmid         int64  
 8   geometry      object 
 9   PFU_1in50     float64
 10  t50           float64
dtypes: float64(4), int64(5), object(2)
memory usage: 546.3+ MB
None


In [29]:
print('start: %s\n' % time.ctime())
gTime = gn.edges_and_nodes_gdf_to_graph(nodes, floodjoin, node_tag='node_ID', u_tag='stnode', v_tag='endnode', geometry_tag='geometry')
gn.example_edge(gTime, 10)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Fri Jan  7 23:17:29 2022

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000021417A4E820>, 'Unnamed: 0': 0, 'Unnamed: 0.1': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in50': -1.0, 't50': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000021417A4EBE0>, 'Unnamed: 0': 1, 'Unnamed: 0.1': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000021417A4E6A0>, 'Unnamed: 0': 2, 'Unnamed: 0.1': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000021417A4E310>, 'Unnamed: 0': 3, 'Unnamed: 0.1': 3, 'time': 2.38514400000000

In [30]:
print('start: %s\n' % time.ctime())
gn.save(gTime, 'gTime_pre-project_flood50', out_pth, edges = False, nodes = False)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Fri Jan  7 23:32:31 2022


end: Fri Jan  7 23:35:32 2022

--- processing complete


### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

Already measured distance from origin/destination to nearest node in pre-project script.

In [31]:
#%% If starting new session, reload graph from file
del gTime, floodjoin, nodes
print(dir())
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime_pre-project_flood50.pickle")
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_24', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'exit', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']
['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_24', '_7', '_8', '_9', '__', '___', '__builtin__'

In [32]:
# If starting a new session, load from file.
HDurban_snap = os.path.join(out_pth, "HDurban_snap.csv")
HDurban_snap = pd.read_csv(HDurban_snap)
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv")
hamlet_snap = pd.read_csv(hamlet_snap)

In [33]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)

In [34]:
HDurban_snap

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,AREA_GEO,SUM,Total_Driv,WP_dens,urb,hd_urb,Urb_class,Shape_Leng,Shape_Area,geometry,NN,NN_dist
0,0,1,28PCU6189_01,bua,27.151434,1.860466e+05,0.000000,6852.183502,1,1,2,0.671746,2.258828e-03,POINT (-16.27318626010657 12.562367084144856),6058226279,2.216606
1,1,8,28PEV0625_01,bua,25.799549,1.064198e+05,0.000000,4124.872441,1,1,2,0.718941,2.149259e-03,POINT (-14.936187613172205 12.892160925397892),6029307183,45.594167
2,2,37,28PBA8597_01,bua,81.242354,3.382677e+05,0.000000,4163.686114,1,1,2,2.303639,6.813610e-03,POINT (-16.987028969206293 14.445279143289168),4998093094,55.383696
3,3,47,28PCA8365_01,bua,42.134069,2.755403e+05,0.000000,6539.609258,1,1,2,1.070058,3.528274e-03,POINT (-16.076128475299285 14.160382012387636),2201506815,55.754910
4,4,56,28PDA4058_01,bua,11.123881,5.186398e+04,0.000000,4662.399756,1,1,2,0.298223,9.311211e-04,POINT (-15.547596052646174 14.100419699131665),4321656809,61.727308
5,5,75,28PBB9234_01,bua,53.453391,2.911360e+05,0.000000,5446.539082,1,1,2,1.468618,4.489114e-03,POINT (-16.926149059908756 14.780621967102865),1697006012,23.047883
6,6,76,28PBB4632_01,bua,226.850348,3.637718e+06,0.000000,16035.762110,1,1,2,4.771359,1.905124e-02,POINT (-17.35638706164378 14.751962825615575),1901689169,5.826006
7,7,80,28PCB0453_01,bua,19.580942,8.712281e+04,0.000000,4449.367491,1,1,2,0.611345,1.645460e-03,POINT (-16.81720508223022 14.94580992504745),6032060028,88.156089
8,8,84,28PCB6720_01,bua,23.634834,1.342721e+05,0.000000,5681.109779,1,1,2,0.689064,1.983306e-03,POINT (-16.227786747221682 14.653718853747844),6040927878,41.930884
9,9,98,28PDB0443_01,bua,196.802191,9.490037e+05,0.000000,2575.818720,1,1,2,3.094788,1.653195e-02,POINT (-15.887758561636817 14.860301673230607),3449495495,28.431974


In [35]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
list_hamlet = list(hamlet_snap.NN.unique())
origins = list_hamlet

In [36]:
list_ag = list(ag_snap.NN.unique())
originslist = list_hamlet + list_ag
origins = list(set(originslist))

In [37]:
dests = list(HDurban_snap.NN.unique()) 

In [38]:
len(origins) # 985720 unique nearest nodes.

985720

In [39]:
len(dests) # 58 unique nearest nodes. 

58

In [40]:
gn.example_edge(gTime,10)

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000213863B8C70>, 'Unnamed: 0': 0, 'Unnamed: 0.1': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in50': -1.0, 't50': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000213FE5BD1C0>, 'Unnamed: 0': 1, 'Unnamed: 0.1': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000213FE5BD7F0>, 'Unnamed: 0': 2, 'Unnamed: 0.1': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000215DF33F8E0>, 'Unnamed: 0': 3, 'Unnamed: 0.1': 3, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 

calculate_OD won't run if any of the edge times are null or zero. If there are any hits, reassign time values for those edges to a very small time. A more efficient way to do this would be to simplify junctions with the clean_network() tool, but that was throwing errors.

In [41]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [42]:
print('start: %s\n' % time.ctime())
OD = gn.calculate_OD(gTime, origins, dests, fail_value, weight = 't50')
# Takes a few minutes.
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Fri Jan  7 23:40:20 2022


end: Sat Jan  8 00:14:07 2022

--- processing complete


In [43]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [44]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985720 entries, 3331684166 to 3706826877
Data columns (total 58 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   6058226279  985720 non-null  float64
 1   6029307183  985720 non-null  float64
 2   4998093094  985720 non-null  float64
 3   2201506815  985720 non-null  float64
 4   4321656809  985720 non-null  float64
 5   1697006012  985720 non-null  float64
 6   1901689169  985720 non-null  float64
 7   6032060028  985720 non-null  float64
 8   6040927878  985720 non-null  float64
 9   3449495495  985720 non-null  float64
 10  3990543961  985720 non-null  float64
 11  8972391475  985720 non-null  float64
 12  3418418812  985720 non-null  float64
 13  1983641803  985720 non-null  float64
 14  6014451367  985720 non-null  float64
 15  6027163276  985720 non-null  float64
 16  2833577858  985720 non-null  float64
 17  7321088861  985720 non-null  float64
 18  6045659373  985720 non-null  fl

In [45]:
OD_df.tail()

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
8925478824,374622.993017,534714.687089,6294.513822,10817.165367,344976.431423,4119.039466,6514.333964,2932.849334,118766.313972,5300.161107,...,9043.084400,10281.605269,454892.011369,454874.412607,6547.461643,6106.681600,6074.985768,6069.733378,5435.576714,5485.793684
3706826868,302145.220413,462236.914485,615052.559622,606753.422529,264314.007033,612986.329720,615466.766349,611888.914646,714702.571134,604023.032339,...,616979.309359,618217.830228,382414.238764,382396.640002,615305.507444,615059.113985,615027.418152,615022.165763,604231.839749,604643.615459
3706826871,232760.161467,392851.855539,545667.500676,537368.363583,194928.948087,543601.270774,546081.707403,542503.855700,645317.512188,534637.973393,...,547594.250413,548832.771282,313029.179818,313011.581056,545920.448498,545674.055039,545642.359206,545637.106817,534846.780803,535258.556513
3706826876,232424.189685,392515.883757,545331.528894,537032.391801,194592.976305,543265.298992,545745.735621,542167.883918,644981.540406,534302.001611,...,547258.278631,548496.799500,312693.208036,312675.609274,545584.476716,545338.083257,545306.387424,545301.135035,534510.809021,534922.584731
3706826877,232775.276715,392866.970787,545682.615924,537383.478831,194944.063335,543616.386022,546096.822651,542518.970948,645332.627436,534653.088641,...,547609.365661,548847.886530,313044.295066,313026.696304,545935.563746,545689.170287,545657.474454,545652.222065,534861.896051,535273.671761


In [46]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(out_pth, 'OD_allorigins_pre-project_flood50.csv'))
OD_min

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3331684166,6213.530255,8881.725156,288.418746,11.536460,5719.420895,253.981581,295.322191,235.691330,1869.250269,104.593291,...,320.531242,341.173256,7551.347228,7551.053915,292.634543,288.527985,287.999722,287.912182,108.073415,114.936343
3571449893,520.662115,3042.737410,5966.540964,5828.222012,257.309284,5932.103799,5973.444409,5913.813547,7627.374489,5782.715509,...,5998.653459,6019.295474,1712.359481,1712.066169,5970.756761,5966.650203,5966.121939,5966.034399,5786.195632,5793.058561
3571449966,521.049027,3043.124322,5966.927875,5828.608924,257.696195,5932.490710,5973.831321,5914.200459,7627.761400,5783.102420,...,5999.040371,6019.682385,1712.746393,1712.453080,5971.143672,5967.037115,5966.508851,5966.421311,5786.582544,5793.445472
3405774993,396.351017,3178.487893,5956.171840,5817.852888,246.940160,5921.734675,5963.075285,5903.444424,7617.005365,5772.346385,...,5988.284336,6008.926350,1848.109964,1847.816651,5960.387637,5956.281079,5955.752815,5955.665276,5775.826509,5782.689437
3405774994,395.950268,3178.087144,5955.771091,5817.452139,246.539411,5921.333926,5962.674536,5903.043675,7616.604616,5771.945636,...,5987.883587,6008.525601,1847.709215,1847.415902,5959.986888,5955.880330,5955.352066,5955.264527,5775.425760,5782.288688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8925478824,6243.716550,8911.911451,104.908564,180.286089,5749.607190,68.650658,108.572233,48.880822,1979.438566,88.336018,...,150.718073,171.360088,7581.533523,7581.240210,109.124361,101.778027,101.249763,101.162223,90.592945,91.429895
3706826868,5035.753674,7703.948575,10250.875994,10112.557042,4405.233451,10216.438829,10257.779439,10198.148577,11911.709519,10067.050539,...,10282.988489,10303.630504,6373.570646,6373.277333,10255.091791,10250.985233,10250.456969,10250.369429,10070.530662,10077.393591
3706826871,3879.336024,6547.530926,9094.458345,8956.139393,3248.815801,9060.021180,9101.361790,9041.730928,10755.291870,8910.632890,...,9126.570840,9147.212855,5217.152997,5216.859684,9098.674142,9094.567584,9094.039320,9093.951780,8914.113013,8920.975942
3706826876,3873.736495,6541.931396,9088.858815,8950.539863,3243.216272,9054.421650,9095.762260,9036.131399,10749.692340,8905.033360,...,9120.971311,9141.613325,5211.553467,5211.260155,9093.074612,9088.968054,9088.439790,9088.352251,8908.513484,8915.376412


In [47]:
print(OD_min.isna().sum())

6058226279    3
6029307183    3
4998093094    3
2201506815    3
4321656809    3
1697006012    3
1901689169    3
6032060028    3
6040927878    3
3449495495    3
3990543961    3
8972391475    3
3418418812    3
1983641803    3
6014451367    3
6027163276    3
2833577858    3
7321088861    3
6045659373    3
5528866190    3
6027939517    3
6031093257    3
1883155712    3
6026417971    3
6031245697    3
7493947593    3
1859359090    3
6053816462    3
6027530676    3
2988435647    3
7838754747    3
6053825179    3
2988435792    3
3036877534    3
6056472223    3
7460241128    3
6028595154    3
4053739078    3
4071261250    3
6014815775    3
6024430122    3
8229317581    3
6027834438    3
6024894515    3
6024894530    3
6024894513    3
8169321141    3
8200304626    3
1968458114    3
9359670710    3
6021177135    3
6027615161    3
6027276892    3
6041228287    3
5536661253    3
7357630367    3
8178147277    3
6026834850    3
dtype: int64


In [48]:
# Create origin-specific matrix and save to file.
OD_hamlet = OD_df.loc[list_hamlet,:]
OD_hamlet = OD_hamlet[OD_hamlet < fail_value] / 60 
OD_hamlet.to_csv(os.path.join(out_pth, 'OD_hamlet_pre-project_flood50.csv'))
OD_hamlet

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
7761872870,7555.041976,11114.143037,13891.826984,13753.508032,8182.595304,13857.389819,13898.730429,13839.099568,15552.660509,13708.001529,...,13923.939479,13944.581494,9783.765108,9783.471795,13896.042781,13891.936223,13891.407959,13891.320420,13711.481653,13718.344581
7761872869,7555.023417,11114.124477,13891.808424,13753.489473,8182.576744,13857.371259,13898.711870,13839.081008,15552.641950,13707.982970,...,13923.920920,13944.562934,9783.746548,9783.453236,13896.024221,13891.917664,13891.389400,13891.301860,13711.463093,13718.326022
6442044321,7554.013585,11113.114645,13890.798592,13752.479641,8181.566912,13856.361427,13897.702038,13838.071176,15551.632118,13706.973138,...,13922.911088,13943.553102,9782.736716,9782.443404,13895.014389,13890.907832,13890.379568,13890.292028,13710.453261,13717.316190
2142496418,7555.467107,11114.568167,13892.252114,13753.933163,8183.020434,13857.814949,13899.155560,13839.524698,15553.085640,13708.426660,...,13924.364610,13945.006624,9784.190238,9783.896926,13896.467911,13892.361354,13891.833090,13891.745550,13711.906783,13718.769712
2142496429,7555.849481,11114.950541,13892.634488,13754.315537,8183.402808,13858.197323,13899.537934,13839.907072,15553.468014,13708.809034,...,13924.746984,13945.388998,9784.572612,9784.279300,13896.850285,13892.743728,13892.215464,13892.127924,13712.289157,13719.152086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9208004175,407808.542291,410476.737192,401657.732281,401745.111830,407314.432931,401602.316288,401642.237863,401583.581214,403544.264307,401653.161759,...,401498.200522,401426.662175,409146.359264,409146.065951,401661.948078,401635.443657,401634.915393,401634.827853,401655.418686,401656.255636
3884280043,7782.266869,10121.321138,3678.828061,3580.973513,7518.914037,3641.568467,3681.490042,3622.800319,5380.125990,3489.544372,...,3486.750734,3560.877505,5560.727839,5560.434526,3683.043858,3674.695836,3674.167572,3674.080032,3498.737171,3491.253225
3884280001,7775.404642,10114.458911,3671.965834,3574.111286,7512.051810,3634.706240,3674.627815,3615.938092,5373.263763,3482.682145,...,3479.888507,3554.015278,5553.865612,5553.572299,3676.181631,3667.833609,3667.305345,3667.217805,3491.874944,3484.390998
8592243457,7768.998180,10108.052450,3647.759412,3549.904864,7505.645348,3610.499818,3650.421393,3591.731670,5349.057341,3458.475722,...,3455.682085,3529.808856,5547.459150,5547.165838,3651.975209,3643.627187,3643.098923,3643.011383,3467.668522,3460.184575


In [49]:
print(OD_hamlet.isna().sum())

6058226279    1
6029307183    1
4998093094    1
2201506815    1
4321656809    1
1697006012    1
1901689169    1
6032060028    1
6040927878    1
3449495495    1
3990543961    1
8972391475    1
3418418812    1
1983641803    1
6014451367    1
6027163276    1
2833577858    1
7321088861    1
6045659373    1
5528866190    1
6027939517    1
6031093257    1
1883155712    1
6026417971    1
6031245697    1
7493947593    1
1859359090    1
6053816462    1
6027530676    1
2988435647    1
7838754747    1
6053825179    1
2988435792    1
3036877534    1
6056472223    1
7460241128    1
6028595154    1
4053739078    1
4071261250    1
6014815775    1
6024430122    1
8229317581    1
6027834438    1
6024894515    1
6024894530    1
6024894513    1
8169321141    1
8200304626    1
1968458114    1
9359670710    1
6021177135    1
6027615161    1
6027276892    1
6041228287    1
5536661253    1
7357630367    1
8178147277    1
6026834850    1
dtype: int64


In [50]:
# Create origin-specific matrix and save to file.
OD_ag = OD_df.loc[list_ag,:]
OD_ag = OD_ag[OD_ag < fail_value] / 60 
OD_ag.to_csv(os.path.join(out_pth, 'OD_ag_pre-project_flood50.csv'))
OD_ag

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3507831609,84.063858,3644.738929,6422.422876,6284.103925,713.191196,6387.985711,6429.326322,6369.695460,8083.256401,6238.597421,...,6454.535372,6475.177386,2314.361000,2314.067687,6426.638673,6422.532115,6422.003852,6421.916312,6242.077545,6248.940473
3507831510,90.422354,3651.097425,6428.781373,6290.462421,719.549692,6394.344207,6435.684818,6376.053956,8089.614898,6244.955918,...,6460.893868,6481.535883,2320.719497,2320.426184,6432.997170,6428.890612,6428.362348,6428.274808,6248.436041,6255.298970
6188134127,25.237636,3585.912707,6363.596654,6225.277703,654.364974,6329.159489,6370.500100,6310.869238,8024.430179,6179.771199,...,6395.709150,6416.351164,2255.534778,2255.241465,6367.812451,6363.705893,6363.177630,6363.090090,6183.251323,6190.114251
8631201421,1070.956229,2782.475931,6516.835077,6378.516125,807.603397,6482.397912,6523.738522,6464.107661,8177.668602,6333.009622,...,6548.947573,6569.589587,2079.632570,2079.339257,6521.050874,6516.944316,6516.416053,6516.328513,6336.489746,6343.352674
8598305977,1085.282837,2796.802539,6531.161686,6392.842734,821.930005,6496.724521,6538.065131,6478.434269,8191.995211,6347.336231,...,6563.274181,6583.916196,2093.959178,2093.665866,6535.377483,6531.270925,6530.742661,6530.655121,6350.816354,6357.679283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651042474,297700.252222,300368.447123,291549.442212,291636.821761,297206.142862,291494.026219,291533.947794,291475.291145,293435.974238,291544.871690,...,291389.910453,291318.372106,299038.069195,299037.775882,291553.658009,291527.153588,291526.625324,291526.537784,291547.128617,291547.965567
3651042508,297692.101967,300360.296868,291541.291958,291628.671506,297197.992607,291485.875964,291525.797539,291467.140890,293427.823983,291536.721435,...,291381.760198,291310.221851,299029.918940,299029.625627,291545.507755,291519.003333,291518.475069,291518.387529,291538.978362,291539.815312
3651042501,297697.597083,300365.791984,291546.787073,291634.166622,297203.487723,291491.371079,291531.292654,291472.636005,293433.319099,291542.216551,...,291387.255313,291315.716967,299035.414055,299035.120743,291551.002870,291524.498448,291523.970184,291523.882645,291544.473478,291545.310427
3651042393,297684.066780,300352.261682,291533.256771,291620.636320,297189.957421,291477.840777,291517.762352,291459.105703,293419.788796,291528.686249,...,291373.725011,291302.186665,299021.883753,299021.590440,291537.472568,291510.968146,291510.439882,291510.352342,291530.943175,291531.780125


In [51]:
print(OD_ag.isna().sum())

6058226279    2
6029307183    2
4998093094    2
2201506815    2
4321656809    2
1697006012    2
1901689169    2
6032060028    2
6040927878    2
3449495495    2
3990543961    2
8972391475    2
3418418812    2
1983641803    2
6014451367    2
6027163276    2
2833577858    2
7321088861    2
6045659373    2
5528866190    2
6027939517    2
6031093257    2
1883155712    2
6026417971    2
6031245697    2
7493947593    2
1859359090    2
6053816462    2
6027530676    2
2988435647    2
7838754747    2
6053825179    2
2988435792    2
3036877534    2
6056472223    2
7460241128    2
6028595154    2
4053739078    2
4071261250    2
6014815775    2
6024430122    2
8229317581    2
6027834438    2
6024894515    2
6024894530    2
6024894513    2
8169321141    2
8200304626    2
1968458114    2
9359670710    2
6021177135    2
6027615161    2
6027276892    2
6041228287    2
5536661253    2
7357630367    2
8178147277    2
6026834850    2
dtype: int64


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [52]:
del gTime, ag_snap, hamlet_snap, HDurban_snap, OD, OD_df, OD_min
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'OD_ag', 'OD_hamlet', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_24', '_34', '_38', '_39', '_45', '_46', '_48', '_50', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i4', '_i40', '_i41', '_i42', '_i43', '_i44', '_i45', '_i46', '_i47', '_i48', '_i49', '_i5', '_i50', '_i51', '_i52', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'dests', 'exit', 'fail_value', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'list_ag', 'list_hamlet', 'loads', 'losm', 'np', 'nx', 'origins', 'originslist', 'os', 'out_pth', 'ox', 'pc_f

In [53]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
OD_hamlet = os.path.join(out_pth, "OD_hamlet_pre-project_flood50.csv")
OD_hamlet = pd.read_csv(OD_hamlet)

In [54]:
OD_ag = os.path.join(out_pth, "OD_ag_pre-project_flood50.csv")
OD_ag = pd.read_csv(OD_ag)

In [55]:
OD_ag.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 

In [56]:
OD_hamlet.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 

In [57]:
OD_ag

,NN,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,3507831609,84.063858,3644.738929,6422.422876,6284.103925,713.191196,6387.985711,6429.326322,6369.695460,8083.256401,...,6454.535372,6475.177386,2314.361000,2314.067687,6426.638673,6422.532115,6422.003852,6421.916312,6242.077545,6248.940473
1,3507831510,90.422354,3651.097425,6428.781373,6290.462421,719.549692,6394.344207,6435.684818,6376.053956,8089.614898,...,6460.893868,6481.535883,2320.719497,2320.426184,6432.997170,6428.890612,6428.362348,6428.274808,6248.436041,6255.298970
2,6188134127,25.237636,3585.912707,6363.596654,6225.277703,654.364974,6329.159489,6370.500100,6310.869238,8024.430179,...,6395.709150,6416.351164,2255.534778,2255.241465,6367.812451,6363.705893,6363.177630,6363.090090,6183.251323,6190.114251
3,8631201421,1070.956229,2782.475931,6516.835077,6378.516125,807.603397,6482.397912,6523.738522,6464.107661,8177.668602,...,6548.947573,6569.589587,2079.632570,2079.339257,6521.050874,6516.944316,6516.416053,6516.328513,6336.489746,6343.352674
4,8598305977,1085.282837,2796.802539,6531.161686,6392.842734,821.930005,6496.724521,6538.065131,6478.434269,8191.995211,...,6563.274181,6583.916196,2093.959178,2093.665866,6535.377483,6531.270925,6530.742661,6530.655121,6350.816354,6357.679283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967547,3651042474,297700.252222,300368.447123,291549.442212,291636.821761,297206.142862,291494.026219,291533.947794,291475.291145,293435.974238,...,291389.910453,291318.372106,299038.069195,299037.775882,291553.658009,291527.153588,291526.625324,291526.537784,291547.128617,291547.965567
967548,3651042508,297692.101967,300360.296868,291541.291958,291628.671506,297197.992607,291485.875964,291525.797539,291467.140890,293427.823983,...,291381.760198,291310.221851,299029.918940,299029.625627,291545.507755,291519.003333,291518.475069,291518.387529,291538.978362,291539.815312
967549,3651042501,297697.597083,300365.791984,291546.787073,291634.166622,297203.487723,291491.371079,291531.292654,291472.636005,293433.319099,...,291387.255313,291315.716967,299035.414055,299035.120743,291551.002870,291524.498448,291523.970184,291523.882645,291544.473478,291545.310427
967550,3651042393,297684.066780,300352.261682,291533.256771,291620.636320,297189.957421,291477.840777,291517.762352,291459.105703,293419.788796,...,291373.725011,291302.186665,299021.883753,299021.590440,291537.472568,291510.968146,291510.439882,291510.352342,291530.943175,291531.780125


In [58]:
OD_hamlet

,NN,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,7761872870,7555.041976,11114.143037,13891.826984,13753.508032,8182.595304,13857.389819,13898.730429,13839.099568,15552.660509,...,13923.939479,13944.581494,9783.765108,9783.471795,13896.042781,13891.936223,13891.407959,13891.320420,13711.481653,13718.344581
1,7761872869,7555.023417,11114.124477,13891.808424,13753.489473,8182.576744,13857.371259,13898.711870,13839.081008,15552.641950,...,13923.920920,13944.562934,9783.746548,9783.453236,13896.024221,13891.917664,13891.389400,13891.301860,13711.463093,13718.326022
2,6442044321,7554.013585,11113.114645,13890.798592,13752.479641,8181.566912,13856.361427,13897.702038,13838.071176,15551.632118,...,13922.911088,13943.553102,9782.736716,9782.443404,13895.014389,13890.907832,13890.379568,13890.292028,13710.453261,13717.316190
3,2142496418,7555.467107,11114.568167,13892.252114,13753.933163,8183.020434,13857.814949,13899.155560,13839.524698,15553.085640,...,13924.364610,13945.006624,9784.190238,9783.896926,13896.467911,13892.361354,13891.833090,13891.745550,13711.906783,13718.769712
4,2142496429,7555.849481,11114.950541,13892.634488,13754.315537,8183.402808,13858.197323,13899.537934,13839.907072,15553.468014,...,13924.746984,13945.388998,9784.572612,9784.279300,13896.850285,13892.743728,13892.215464,13892.127924,13712.289157,13719.152086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74657,9208004175,407808.542291,410476.737192,401657.732281,401745.111830,407314.432931,401602.316288,401642.237863,401583.581214,403544.264307,...,401498.200522,401426.662175,409146.359264,409146.065951,401661.948078,401635.443657,401634.915393,401634.827853,401655.418686,401656.255636
74658,3884280043,7782.266869,10121.321138,3678.828061,3580.973513,7518.914037,3641.568467,3681.490042,3622.800319,5380.125990,...,3486.750734,3560.877505,5560.727839,5560.434526,3683.043858,3674.695836,3674.167572,3674.080032,3498.737171,3491.253225
74659,3884280001,7775.404642,10114.458911,3671.965834,3574.111286,7512.051810,3634.706240,3674.627815,3615.938092,5373.263763,...,3479.888507,3554.015278,5553.865612,5553.572299,3676.181631,3667.833609,3667.305345,3667.217805,3491.874944,3484.390998
74660,8592243457,7768.998180,10108.052450,3647.759412,3549.904864,7505.645348,3610.499818,3650.421393,3591.731670,5349.057341,...,3455.682085,3529.808856,5547.459150,5547.165838,3651.975209,3643.627187,3643.098923,3643.011383,3467.668522,3460.184575


#### Find first, second, and third nearest destination for each origin node. 

In [59]:
fail_value = 999999999

In [60]:
# Nearest
OD_ag["ag_HD1"] = 0
sub = OD_ag.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_ag["ag_HD1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
ag1 = OD_ag[['NN', 'ag_HD1']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_ag["ag_HD2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_ag["ag_HD2"] = Dsub.min(axis=1) 
ag2 = OD_ag.loc[:,['NN', 'ag_HD2']] 


# Third nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value)

OD_ag["ag_HD3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_ag["ag_HD3"] = Dsub.min(axis=1)
ag3 = OD_ag.loc[:,['NN', 'ag_HD3']]

# Combine and write to file
ag_all = OD_ag.loc[:,['NN', 'ag_HD1', 'ag_HD2', 'ag_HD3']]
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_drive_pre-project_flood50.csv'))
ag_all.head()

,NN,ag_HD1,ag_HD2,ag_HD3
0,3507831609,82.363044,84.063858,687.136217
1,3507831510,88.721541,90.422354,693.494714
2,6188134127,23.536822,25.237636,628.309995
3,8631201421,615.678582,807.603397,1070.956229
4,8598305977,630.005190,821.930005,1085.282837


In [61]:
# Nearest
OD_hamlet["ha_HD1"] = 0
sub = OD_hamlet.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_hamlet["ha_HD1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
hamlet1 = OD_hamlet[['NN', 'ha_HD1']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_hamlet["ha_HD2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_hamlet["ha_HD2"] = Dsub.min(axis=1) 
hamlet2 = OD_hamlet.loc[:,['NN', 'ha_HD2']] 


# Third nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value)
OD_hamlet["ha_HD3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_hamlet["ha_HD3"] = Dsub.min(axis=1)
hamlet3 = OD_hamlet.loc[:,['NN', 'ha_HD3']]


# Combine and write to file
hamlet_all = OD_hamlet.loc[:,['NN', 'ha_HD1', 'ha_HD2', 'ha_HD3']]
hamlet_all.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_drive_pre-project_flood50.csv'))
hamlet_all.head()

,NN,ha_HD1,ha_HD2,ha_HD3
0,7761872870,7555.041976,7558.410156,8156.540325
1,7761872869,7555.023417,7558.391596,8156.521766
2,6442044321,7554.013585,7557.381764,8155.511934
3,2142496418,7555.467107,7558.835286,8156.965456
4,2142496429,7555.849481,7559.217660,8157.347830


### End of script. Load into QGIS or Arc and visualize at 10 min intervals. 
QML file for symbology in QGIS:
R:\GEOGlobal\Design\symb_traveltimes_10min.qml